In [1]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import default_rng
import itertools as it
import sys
import progressbar
import time
import random

from joblib import Parallel, delayed

from scipy import linalg
from scipy.linalg import LinAlgWarning

import pickle

warnings.filterwarnings(action='error', category=LinAlgWarning, module='sklearn')

def stack_xy(A):
    return np.block([[A["xx"], A["xy"]], 
                    [A["xy"], A["yy"]]])

def restructure_dict(list_dic, shape):
    keys = list( list_dic[0].keys() )

    new_dic = dict()
    
    for key in keys:
        new_dic[key] = []
    
    for item in list_dic:
        for key in keys:
            new_dic[key].append(item[key]) 

    for key in keys:
        arr = np.asarray(new_dic[key])
        new_dic[key] = arr.reshape(shape)
        
    return new_dic

In [6]:
def get_r_matrix(v_x,v_y):
    
    X, Y = np.meshgrid( v_x, v_y )
    X = np.reshape( np.array(X), -1)
    Y = np.reshape( np.array(Y), -1)
    r = np.zeros((X.size,Y.size))
    for i, (xi, yi) in enumerate(zip(X, Y)):
        for j, (xj, yj) in enumerate(zip(X, Y)):
            r[i,j] = np.sqrt((xi-xj)**2 + (yi-yj)**2)
            
    return r
    
    

def create_G_2D(v_x, v_y, rz=0., k=2.*np.pi, thres_same=0.1, thres_ren=1e-12):
            
    X, Y = np.meshgrid( v_x, v_y )
    X = np.reshape( np.array(X), -1)
    Y = np.reshape( np.array(Y), -1)
    
    rf = np.zeros((X.size,Y.size))
    rx = np.zeros((X.size,Y.size))
    ry = np.zeros((X.size,Y.size))
    for i, (xi, yi) in enumerate(zip(X, Y)):
        for j, (xj, yj) in enumerate(zip(X, Y)):
            rf[i,j] = np.sqrt((xi-xj)**2 + (yi-yj)**2)
            rx[i,j] = np.abs(xi-xj)
            ry[i,j] = np.abs(yi-yj)

    # add some random noise for divergences
    r = rf + thres_ren*np.random.random_sample(rf.shape)    
    
    phase = np.exp(1j*k*r)/(4*np.pi*r)

    G = dict()
    G["xx"] = phase * ( 1. + (1j*k*r-1.)/(k*r)**2 + ( -1. + (3.-3.*1j*k*r)/(k*r)**2) * rx**2/r**2 ) 
    G["yy"] = phase * ( 1. + (1j*k*r-1.)/(k*r)**2 + ( -1. + (3.-3.*1j*k*r)/(k*r)**2) * ry**2/r**2 )
    G["xy"] = phase * ( ( -1. + (3.-3.*1j*k*r)/(k*r)**2) * rx*ry/r**2 )
    
    for key in G.keys():
        G[key][rf<thres_same] = 0. # set vals with r=0. to zero (don't count emitter interacting with itself)

    return G

def create_alpha_2D(v_x, v_y, delta, d_delta, a, gamma_0, gamma_nr=0., lambda_a=1., epsilon=12.25, theta=0.):
    
    gamma = gamma_0 + gamma_nr
    
    X, Y = np.meshgrid( v_x, v_y )
    X = np.reshape( np.array(X), -1)
    Y = np.reshape( np.array(Y), -1)
    
    alpha = np.zeros((X.size,Y.size), dtype=complex)
    
    for i, (xi, yi) in enumerate(zip(X, Y)):
        for j, (xj, yj) in enumerate(zip(X, Y)):       
            delta_r = delta + d_delta*np.sin(2.*np.pi*(np.cos(theta)*xi + np.sin(theta)*yi)/a)
            alpha[i,j] = -3./(4.*np.pi**2)*epsilon*lambda_a**3*(gamma_0/2.)/(delta_r+1j*gamma/2.)
            
    return alpha

In [7]:
# single case test

# system parameters
gamma_0 = 2.
gamma_nr = .0
delta = 0.
d_delta = 1.
n_exc = 5.

a = 0.2
theta = 0.

x_max = 1.
N_x = 50
y_max = 1.
N_y = 10

########
v_x = np.linspace(0.,x_max, N_x)
v_y = np.linspace(0., y_max, N_y)

Gd = create_G_2D(v_x, v_y)
G = stack_xy(Gd)

alpha = create_alpha_2D(v_x, v_y, delta, d_delta, a, gamma_0, gamma_nr=0., theta=0.)

p0 = 

In [15]:
v_y

array([0.])